In [ ]:
from functools import lru_cache
import sys
sys.setrecursionlimit(10**9)
# input = sys.stdin.readline
from decimal import Decimal
from functools import cmp_to_key
from collections import Counter
from itertools import permutations
from itertools import combinations
from itertools import combinations_with_replacement
from itertools import product
from itertools import accumulate
from itertools import groupby
from itertools import pairwise
from copy import deepcopy
import networkx as nx
import networkx.algorithms as nxa
import numpy as np
import math
from heapq import *
from collections import OrderedDict
from bisect import *
from collections import deque

from collections import defaultdict

# セグメント木
# op: op(x,y) return x*y みたいな演算(関数)
# e: 単位元を返す関数，op(x,e()) = op(e(),x) = x になるようなe
# n: 要素数
# v: 要素の配列
class SegTree:
    def __init__(self, op, e, n, v=None):
        # 要素数
        self._n = n
        # 二項演算
        self._op = op
        # 単位元を返す関数
        self._e = e
        # セグメント木の深さ(根は0)
        self._log = (n - 1).bit_length()
        # 葉の数
        self._size = 1 << self._log
        # セグメント木(0番目の要素は使わない，1番目の要素が根)
        self._d = [self._e()] * (self._size << 1)
        if v is not None:
            for i in range(self._n):
                self._d[self._size + i] = v[i]
            for i in range(self._size - 1, 0, -1):
                self._d[i] = self._op(self._d[i << 1], self._d[i << 1 | 1])
    # p番目の要素をxに変更(pは0-indexed)
    def set(self, p, x):
        p += self._size
        self._d[p] = x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素にxを加算(pは0-indexed)
    def add(self, p, x):
        p += self._size
        self._d[p] += x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素を返す(pは0-indexed)
    def get(self, p):
        return self._d[p + self._size]
    # op(d[l], d[l+1], ..., d[r-1])を返す(l,rは0-indexedの半開区間)
    def prod(self, l, r):
        sml, smr = self._e(), self._e()
        l += self._size
        r += self._size
        while l < r:
            if l & 1:
                sml = self._op(sml, self._d[l])
                l += 1
            if r & 1:
                r -= 1
                smr = self._op(self._d[r], smr)
            l >>= 1
            r >>= 1
        return self._op(sml, smr)
    # op(d[0], d[1], ..., d[n-1])を返す
    def all_prod(self):
        return self._d[1]
    # _d[1] = d[i]となる最小のi-self.size(0-indexedのノード番号)を返す
    def min_(self):
        # 深さ
        l = 0
        ans_idx = 1
        while l < self._log:
            # print(f"ans_idx:{ans_idx}")
            # 左の子と右の子
            left = ans_idx << 1
            right = left + 1
            # 左の子が
            if self._d[left] == self._d[ans_idx]:
                ans_idx = left
            else:
                ans_idx = right
            l += 1
        return ans_idx - self._size

def main():
    N,M = map(int,input().split())
    A = list(map(int,input().split()))
    # 各頂点に隣接する頂点のセット
    adj_dict = defaultdict(set)
    # 各頂点が隣接する頂点のコストの総和の初期値
    adj_cost = [0] * N
    for _ in range(M):
        u,v = map(int,input().split())
        adj_dict[u].add(v)  
        adj_dict[v].add(u)
        adj_cost[u-1] += A[v-1]
        adj_cost[v-1] += A[u-1]
    # 各頂点が隣接する頂点のコストの総和を持つセグメント木
    seg = SegTree(min, lambda:10**16, N, adj_cost)
    ans = 0
    for _ in range(N):
        # コストが最小の頂点を選択
        # print(f"seg._d:{seg._d}")
        u = seg.min_()
        # print(f"u:{u}")
        ans = max(ans, seg.get(u))
        # print(f"ans:{ans}")
        # 選択した頂点を削除
        seg.set(u, 10**16)
        # 選択した頂点と隣接する頂点のコストの総和を更新
        for v in adj_dict[u+1]:
            seg.set(v-1, seg.get(v-1) - A[u])
            adj_dict[v].remove(u+1)
        adj_dict.pop(u+1)
    print(ans)
    
if __name__ == "__main__":
    main()